In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as pltf
from ipywidgets import interact, Dropdown

def read_files(folder_path):
    """
    Reads all the files in the given folder path and returns a dictionary with
    the file names as keys and the file contents as values.
    """
    files = {}
    for filepath in glob.glob(os.path.join(folder_path, "*.txt")):
        with open(filepath, "r", encoding="iso-8859-1") as file:
            files[os.path.basename(filepath)] = file.read()
    return files

def parse_file(file_contents):
    """
    Parses the given file contents and returns a Pandas DataFrame with the x, y pairs.
    """
    data_lines = file_contents.split("\n")[28:]  # Skip the header lines
    data_lines = [line for line in data_lines if line]  # Remove empty lines
    data = [line.split("\t") for line in data_lines]
    df = pd.DataFrame(data, columns=["x", "y"])
    df["x"] = pd.to_numeric(df["x"])
    df["y"] = pd.to_numeric(df["y"])
    return df

def denoise_and_plot(file_contents):
    """
    Applies several denoise methods to the given file contents, plots the results
    and returns a Pandas DataFrame with the denoised x, y pairs.
    """
    df = parse_file(file_contents)
    
    # Apply denoise methods
    df["denoised1"] = df["y"].rolling(window=5, center=True).mean()  # Moving average
    df["denoised2"] = df["y"].rolling(window=5, center=True).median()  # Moving median
    df["denoised3"] = df["y"].apply(lambda x: 0 if x < 0 else x)  # Replace negative values with 0
    
    # Plot the results
    plt.plot(df["x"], df["y"], label="Original")
    plt.plot(df["x"], df["denoised1"], label="Moving average")
    plt.plot(df["x"], df["denoised2"], label="Moving median")
    plt.plot(df["x"], df["denoised3"], label="Replace negatives with 0")
    plt.legend()
    plt.show()
    
    return df[["x", "denoised1", "denoised2", "denoised3"]]

# Define the folder path
folder_path = "raman_2/"

# Read all the files in the folder
files = read_files(folder_path)

# Create a dropdown menu to select the file to plot
file_dropdown = Dropdown(options=files.keys(), description="Select a file:")

# Define the function to call when the dropdown value changes
def on_file_dropdown_change(change):
    file_contents = files[change.new]
    denoise_and_plot(file_contents)

file_dropdown.observe(on_file_dropdown_change, names="value")

# Display the dropdown menu
display(file_dropdown)


Dropdown(description='Select a file:', options=('Si_nano_muestra 3_25.txt', 'Si_nano_muestra 3_16.txt', 'Si_na…

ValueError: Unable to parse string "#Acquired=" at position 0

ValueError: Unable to parse string "#Acquired=" at position 0